Predicts the volume of bikes needed on various days in May 2021. Uses this dataset: https://www.kaggle.com/datasets/farjadanalytica/divvytripdata-1?select=202105-divvy-tripdata.csv 

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
import urllib.request
import csv
import codecs
from scipy import stats

In [19]:
df_divvy = pd.read_csv('all-months-divvy-tripdata.csv', on_bad_lines='skip')
df_divvy[:20]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,s,Unnamed: 13
0,0,B9F73448DFBE0D45,classic_bike,1/24/2021 19:15,1/24/2021 19:22,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,member,NaN,NaN
1,1,457C7F4B5D3DA135,electric_bike,1/23/2021 12:57,1/23/2021 13:02,California Ave & Cortez St,17660,California Ave & North Ave,13258,41.900406,-87.696733,41.910435,-87.696890,member,NaN,NaN
2,2,57C750326F9FDABE,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900374,-87.696688,41.899180,-87.672178,casual,NaN,NaN
3,3,4D518C65E338D070,electric_bike,1/9/2021 15:28,1/9/2021 15:37,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900379,-87.696716,41.899149,-87.672177,casual,NaN,NaN
4,4,9D08A3AFF410474D,classic_bike,1/24/2021 15:56,1/24/2021 16:07,California Ave & Cortez St,17660,Wood St & Augusta Blvd,657,41.900363,-87.696704,41.899181,-87.672200,casual,NaN,NaN
5,5,49FCE1F8598F12C6,electric_bike,1/22/2021 15:15,1/22/2021 15:36,California Ave & Cortez St,17660,Wells St & Elm St,KA1504000135,41.900373,-87.696790,41.903272,-87.634457,member,NaN,NaN
6,6,0FEED5C2C8749A1C,classic_bike,1/5/2021 10:33,1/5/2021 10:39,California Ave & Cortez St,17660,Sacramento Blvd & Franklin Blvd,KA1504000113,41.900363,-87.696704,41.890469,-87.702608,member,NaN,NaN
7,7,E276FD43BDED6420,classic_bike,1/30/2021 11:59,1/30/2021 12:03,California Ave & Cortez St,17660,Western Ave & Walton St,KA1504000103,41.900363,-87.696704,41.898418,-87.686596,member,NaN,NaN
8,8,88BFCF66C2D585EC,electric_bike,1/27/2021 7:27,1/27/2021 7:45,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900313,-87.696792,41.931841,-87.677814,member,NaN,NaN
9,9,8BD6F6510F5C8BD2,electric_bike,1/15/2021 8:54,1/15/2021 9:11,California Ave & Cortez St,17660,Damen Ave & Clybourn Ave,13271,41.900356,-87.696627,41.931922,-87.677856,member,NaN,NaN


In [20]:
# Get historical weather data
weather_df = pd.read_csv('chicago_weather_data.csv')
weather_df.loc[weather_df['datetime'] == '2020-05-01']

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"chicago, illinois",2020-05-01,66.6,38.6,53.6,66.6,38.6,53.4,35.6,54.7,...,22.7,9,NaN,2020-05-01T05:46:00,2020-05-01T19:49:49,0.28,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,74466504838,..."


In [21]:
# Merge weather data and Divvy dataset
df_divvy['datetime'] = df_divvy.apply(lambda x : x['started_at'].split(' ')[0], axis=1)
df_divvy['datetime'] = df_divvy['datetime'].apply(lambda x : x.split('/')[2] + '-' + ('0' + x.split('/')[0])[-2:] + '-' + ('0' + x.split('/')[1])[-2:])
print(df_divvy.head())
#df_divvy['temperature'] = df_divvy.apply(lambda x : weather_df.loc[weather_df['datetime'] == x['started_at']]['temp'], axis=1)
#df_divvy_weather = pd.merge(df_divvy, weather_df, on='datetime')
#df_divvy.merge(weather_df, on='datetime', how='left')
df_divvy_weather = pd.merge(df_divvy, weather_df, on='datetime', how='left')
df_divvy_weather[['started_at', 'datetime', 'temp', 'precip', 'windspeed']].iloc[[1,3000,5000,7000,10000,15000,20000]]
print(len(df_divvy))
print(len(df_divvy_weather))

   Unnamed: 0           ride_id  rideable_type       started_at  \
0           0  B9F73448DFBE0D45   classic_bike  1/24/2021 19:15   
1           1  457C7F4B5D3DA135  electric_bike  1/23/2021 12:57   
2           2  57C750326F9FDABE  electric_bike   1/9/2021 15:28   
3           3  4D518C65E338D070  electric_bike   1/9/2021 15:28   
4           4  9D08A3AFF410474D   classic_bike  1/24/2021 15:56   

          ended_at          start_station_name start_station_id  \
0  1/24/2021 19:22  California Ave & Cortez St            17660   
1  1/23/2021 13:02  California Ave & Cortez St            17660   
2   1/9/2021 15:37  California Ave & Cortez St            17660   
3   1/9/2021 15:37  California Ave & Cortez St            17660   
4  1/24/2021 16:07  California Ave & Cortez St            17660   

             end_station_name end_station_id  start_lat  start_lng    end_lat  \
0      Wood St & Augusta Blvd            657  41.900363 -87.696704  41.899181   
1  California Ave & North Ave   

In [22]:
# Input: ride and weather data. Output: features for which p value for chi2 is less than 0.05
def chi2_test(df):
  ridership_cutoff = 1000 # less than this is fewer rides, more than this is more rides
  features_cutoffs = {'weekend_or_holiday': '>0.01', 'temp': '>50', 'precip': '<0.01', 'windspeed': '<20', 'snowdepth': '<0.01', 'humidity': '<65', 'feelslike': '>50'} # key: feature, value: rule for distinguishing good weather from bad weather for that feature
  influential_features = {} # key: feature, value: p value. Only includes features with p value less than 0.05
  ridershiphigh_weathergood, ridershiphigh_weatherbad, ridershiplow_weathergood, ridershiplow_weatherbad = None, None, None, None

  for feature in list(features_cutoffs.keys()):
    feature_cutoff = float(features_cutoffs[feature][1:])
    if features_cutoffs[feature][0] == '>':
      ridershiphigh_weathergood = len(df[(df['rides_count']>=ridership_cutoff)&(df[feature]>=feature_cutoff)])
      ridershiphigh_weatherbad = len(df[(df['rides_count']>=ridership_cutoff)&(df[feature]<feature_cutoff)])
      ridershiplow_weathergood = len(df[(df['rides_count']<ridership_cutoff)&(df[feature]>=feature_cutoff)])
      ridershiplow_weatherbad = len(df[(df['rides_count']<ridership_cutoff)&(df[feature]<feature_cutoff)])
    elif features_cutoffs[feature][0] == '<':
      ridershiphigh_weathergood = len(df[(df['rides_count']>=ridership_cutoff)&(df[feature]<=feature_cutoff)])
      ridershiphigh_weatherbad = len(df[(df['rides_count']>=ridership_cutoff)&(df[feature]>feature_cutoff)])
      ridershiplow_weathergood = len(df[(df['rides_count']<ridership_cutoff)&(df[feature]<=feature_cutoff)])
      ridershiplow_weatherbad = len(df[(df['rides_count']<ridership_cutoff)&(df[feature]>feature_cutoff)])
    table = [[ridershiphigh_weathergood, ridershiphigh_weatherbad],[ridershiplow_weathergood, ridershiplow_weatherbad]]
    p = stats.chi2_contingency(table)[1]
    if p < 0.05:
      influential_features[feature] = p
  
  return influential_features
    

In [23]:
# Find total number of bikes used on each day
ride_date_counts = {} # key: ride_date, value: number of rides on that date
ride_date_weather = {} # key: ride_date, value: [temp, precip, windspeed, snowdepth, humidity, feelslike]
ride_date_weekend_bool = {} # key: ride_date, value: 1 if the date is a Saturday, Sunday, or holiday, else 0
holidays = ['2020-01-01', '2020-01-20', '2020-05-25', '2020-07-03', '2020-09-07', '2020-11-11', '2020-11-26', '2020-12-25',
            '2021-01-01', '2020-01-18', '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-11', '2021-11-25', '2021-12-24', '2021-12-31']
for index, row in df_divvy_weather.iterrows():
    if row['datetime'] not in ride_date_counts.keys():
      ride_date_counts[row['datetime']] = 1
      ride_date_weather[row['datetime']] = [row['temp'], row['precip'], row['windspeed'], row['snowdepth'], row['humidity'], row['feelslike']]
      current_timestamp = pd.Timestamp(row['datetime'])
      ride_date_weekend_bool[row['datetime']] = int(current_timestamp.dayofweek in [5,6] or row['datetime'] in holidays)
    else:
      ride_date_counts[row['datetime']] += 1
#print(ride_date_counts)
#print(ride_date_weather)

ride_data = pd.DataFrame()
ride_data['datetime'] = list(ride_date_counts.keys())
ride_data['rides_count'] = list(ride_date_counts.values())
ride_data['weekend_or_holiday'] = list(ride_date_weekend_bool.values())
ride_data['temp'] = [weather_data[0] for weather_data in list(ride_date_weather.values())]
ride_data['precip'] = [weather_data[1] for weather_data in list(ride_date_weather.values())]
ride_data['windspeed'] = [weather_data[2] for weather_data in list(ride_date_weather.values())]
ride_data['snowdepth'] = [weather_data[3] for weather_data in list(ride_date_weather.values())]
ride_data['humidity'] = [weather_data[4] for weather_data in list(ride_date_weather.values())]
ride_data['feelslike'] = [weather_data[5] for weather_data in list(ride_date_weather.values())]
#print(ride_data)

# Do chi2 test to determine which variables most influence Divvy bike usage
significant_features = chi2_test(ride_data)
print(significant_features)

#x = ride_data[['weekend_or_holiday', 'temp', 'precip', 'windspeed']]
#x = ride_data[['weekend_or_holiday', 'humidity', 'temp', 'windspeed']] # Use features that the chi2 test showed were most important
x = ride_data[['weekend_or_holiday', 'feelslike', 'humidity']]
#x = ride_data[['weekend_or_holiday', 'temp', 'windspeed']]
y = ride_data[['rides_count']]
#print(ride_data)
#print(x)
#print(y)
#print(list(ride_date_counts.keys()) == list(ride_date_weather.keys()) == list(ride_date_weekend_bool.keys()))
"""
plt.figure(figsize=(40, 20))
plt.plot(list(start_date_counts.keys()), list(start_date_counts.values()))
plt.title('Divvy Bike Usage per Day of May 2021')
plt.xlabel('Day')
plt.ylabel('Number of Bikes Used')
plt.savefig('bike_usage_per_day.png')
"""

{'weekend_or_holiday': 0.0019626734867053265, 'temp': 2.363752512013212e-19, 'windspeed': 0.00012224016427590892, 'snowdepth': 2.4591903863883622e-06, 'humidity': 2.9187253410180532e-05, 'feelslike': 1.9129893966139542e-19}


"\nplt.figure(figsize=(40, 20))\nplt.plot(list(start_date_counts.keys()), list(start_date_counts.values()))\nplt.title('Divvy Bike Usage per Day of May 2021')\nplt.xlabel('Day')\nplt.ylabel('Number of Bikes Used')\nplt.savefig('bike_usage_per_day.png')\n"

In [24]:
# Train and test linear regression model

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
coef = linear_model.coef_
intercept = linear_model.intercept_
print(linear_model.score(X_test, y_test)) # R^2
print(coef)
print(intercept)
print(len(x))

0.5078925915239998
[[209.79606842  15.16303739  -6.69928709]]
[223.09632551]
363


In [8]:
# Create plot showing regression line
"""
print(len(np.arange(1,364)))
print(np.sort(y).shape)
#print(np.sort(y, axis=0))
plt.plot(np.arange(1,364), np.sort(y, axis=0), 'o', label='original data')
y_plot = np.sort(intercept[0] + coef[0][0]*x['weekend_or_holiday'] + coef[0][1]*x['humidity'] + coef[0][2]*x['temp'] + coef[0][3]*x['windspeed'])
plt.plot(np.arange(1,364), y_plot, 'r', label='fitted line')
plt.legend()
#plt.show()
plt.xlabel('Weekend/Holiday/Temperature Data')
plt.ylabel('Number of Bikes Used')
plt.title('Linear Regression - Divvy Bike Usage')
plt.savefig('lin_reg_divvy_bikes.png')
"""

"\nprint(len(np.arange(1,364)))\nprint(np.sort(y).shape)\n#print(np.sort(y, axis=0))\nplt.plot(np.arange(1,364), np.sort(y, axis=0), 'o', label='original data')\ny_plot = np.sort(intercept[0] + coef[0][0]*x['weekend_or_holiday'] + coef[0][1]*x['humidity'] + coef[0][2]*x['temp'] + coef[0][3]*x['windspeed'])\nplt.plot(np.arange(1,364), y_plot, 'r', label='fitted line')\nplt.legend()\n#plt.show()\nplt.xlabel('Weekend/Holiday/Temperature Data')\nplt.ylabel('Number of Bikes Used')\nplt.title('Linear Regression - Divvy Bike Usage')\nplt.savefig('lin_reg_divvy_bikes.png')\n"

In [25]:
# Polynomial fit
#polynomial_model = np.poly1d(np.polyfit(x, y, 5))
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X_train) 
# x0, x1, x2, x3, x0^2, x0x1, 
# x0x2, x0x3, x1^2,
print(X_train[0:3])
print(poly_features[0:3])
poly_reg_model = LinearRegression()
poly_reg_model.fit(poly_features, y_train)
coef = poly_reg_model.coef_
intercept = poly_reg_model.intercept_
X_test_features = poly.fit_transform(X_test)
print(poly_reg_model.score(X_test_features, y_test)) # R^2
print(coef)
print(intercept)



     weekend_or_holiday  feelslike  humidity
63                    1       42.0      49.7
352                   0       36.0      82.2
255                   0       68.5      65.1
[[1.00000e+00 4.20000e+01 4.97000e+01 1.00000e+00 4.20000e+01 4.97000e+01
  1.76400e+03 2.08740e+03 2.47009e+03]
 [0.00000e+00 3.60000e+01 8.22000e+01 0.00000e+00 0.00000e+00 0.00000e+00
  1.29600e+03 2.95920e+03 6.75684e+03]
 [0.00000e+00 6.85000e+01 6.51000e+01 0.00000e+00 0.00000e+00 0.00000e+00
  4.69225e+03 4.45935e+03 4.23801e+03]]
0.6362170643422602
[[-30.77577732   8.08828257  49.6651444  -30.77577732   6.85159421
   -1.21970349   0.1790762   -0.1760737   -0.36636562]]
[-1472.36640846]


In [26]:
# Cross validation
cv_results = cross_validate(poly_reg_model, poly_features, y_train, cv=10)
print(cv_results)
print(np.average(cv_results['fit_time']))
print(np.average(cv_results['score_time']))
print(np.average(cv_results['test_score']))

{'fit_time': array([0.00526166, 0.00226474, 0.00217056, 0.00218391, 0.00261998,
       0.00224018, 0.00220966, 0.0021503 , 0.00220346, 0.00219727]), 'score_time': array([0.00167584, 0.00164437, 0.00163531, 0.00168777, 0.00167465,
       0.00167251, 0.00166655, 0.00257039, 0.00166345, 0.00163007]), 'test_score': array([0.52184901, 0.49379196, 0.58357818, 0.52171651, 0.73216032,
       0.72985782, 0.62038441, 0.64707146, 0.67349128, 0.6943514 ])}
0.002550172805786133
0.0017520904541015625
0.6218252329382424


In [11]:
"""
plt.plot(np.arange(1,364), np.sort(y, axis=0), 'o', label='original data')
#y_plot = X_test_features * coef + intercept
#plt.plot(np.arange(1,364), y_plot)
all_features = np.concatenate((poly_features, X_test_features), axis=0)
print(all_features.shape)
print(coef.shape)
plt.plot(np.arange(1,364), sorted(np.matmul(all_features, coef.T) + intercept))
plt.xlabel('Weekend/Holiday/Temperature Data')
plt.ylabel('Number of Bikes Used')
plt.title('Polynomial Regression - Divvy Bike Usage')
plt.savefig('polynomial_reg_divvy_bikes.png')
"""
#print(r2_score(y, polynomial_model(x)))
#print(polynomial_model) # coefficients

"\nplt.plot(np.arange(1,364), np.sort(y, axis=0), 'o', label='original data')\n#y_plot = X_test_features * coef + intercept\n#plt.plot(np.arange(1,364), y_plot)\nall_features = np.concatenate((poly_features, X_test_features), axis=0)\nprint(all_features.shape)\nprint(coef.shape)\nplt.plot(np.arange(1,364), sorted(np.matmul(all_features, coef.T) + intercept))\nplt.xlabel('Weekend/Holiday/Temperature Data')\nplt.ylabel('Number of Bikes Used')\nplt.title('Polynomial Regression - Divvy Bike Usage')\nplt.savefig('polynomial_reg_divvy_bikes.png')\n"

In [27]:
# Bagging Ensemble Method
from sklearn.ensemble import BaggingRegressor
linear_regr_bagging = BaggingRegressor(base_estimator=linear_model, n_estimators=4, random_state=0).fit(X_train, y_train)
linear_regr_bagging.predict(X_test)
print(linear_regr_bagging.score(X_test, y_test))

from sklearn.ensemble import BaggingRegressor
poly_regr_bagging = BaggingRegressor(base_estimator=poly_reg_model, n_estimators=4, random_state=0).fit(poly_features, y_train)
poly_regr_bagging.predict(X_test_features)
print(poly_regr_bagging.score(X_test_features, y_test))

0.4944829359194146
0.6168737011044725


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:429: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


In [28]:
# Stacking
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor
estimators = [('linear', linear_model), ('polynomial', poly_reg_model)]
reg_stacking = StackingRegressor(estimators=estimators).fit(X_train, y_train)
reg_stacking.predict(X_test)
print(reg_stacking.score(X_test, y_test))

0.5067379100467304


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:758: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
# Grid search for bagging and gradient boosting regressor, using both linear and polynomial model
from sklearn.model_selection import GridSearchCV

def regr_grid_search(regr_model, parameters, X_train, y_train, X_test, y_test):
  #regr_optimized = BaggingRegressor()
  clf = GridSearchCV(regr_model, parameters, scoring='r2', cv=5)
  #print(clf.get_params().keys())
  clf.fit(X_train, np.ravel(y_train))
  print(clf.score(X_test, np.ravel(y_test)))
  print(clf.scorer_)
  print(clf.best_score_)
  #print(clf.cv_results_)
  print(clf.best_params_)

# Bagging
parameters = {'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'max_samples':[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
print("linear, bagging")
regr_grid_search(linear_regr_bagging, parameters, X_train, y_train, X_test, y_test)
print("polynomial, bagging")
regr_grid_search(poly_regr_bagging, parameters, poly_features, y_train, X_test_features, y_test)

# Gradient Boosting
parameters = {'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'subsample':[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
print("linear, gradient boosting")
regr_grid_search(GradientBoostingRegressor(), parameters, X_train, y_train, X_test, y_test)
print("polynomial, gradient boosting")
regr_grid_search(GradientBoostingRegressor(), parameters, poly_features, y_train, X_test_features, y_test)

linear, bagging
0.4984981066924562
make_scorer(r2_score)
0.5424024079464067
{'max_samples': 1.0, 'n_estimators': 17}
polynomial, bagging
0.6192182345949764
make_scorer(r2_score)
0.6267980138281123
{'max_samples': 1.0, 'n_estimators': 17}
linear, gradient boosting
0.5491009836788301
make_scorer(r2_score)
0.605574587845498
{'n_estimators': 20, 'subsample': 0.7}
polynomial, gradient boosting
0.5641707903766912
make_scorer(r2_score)
0.608008711911068
{'n_estimators': 20, 'subsample': 0.5}


In [15]:
"""
# Graph results of the best models
# Linear Gradient Boosting: max_samples=1.0, n_estimators=17
gbr = GradientBoostingRegressor(random_state=0, max_samples=1.0, n_estimators=17)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test[1:2])
print(gbr.score(X_test, y_test))

print(np.sort(y_pred).shape)
#print(np.sort(y, axis=0))
plt.plot(np.arange(1,364), np.sort(y_pred, axis=0), 'o', label='original data')
y_plot = np.sort(intercept[0] + coef[0][0]*x['weekend_or_holiday'] + coef[0][1]*x['humidity'] + coef[0][2]*x['temp'] + coef[0][3]*x['windspeed'])
plt.plot(np.arange(1,364), y_plot, 'r', label='fitted line')
plt.legend()
#plt.show()
plt.xlabel('Weekend/Holiday/Temperature Data')
plt.ylabel('Number of Bikes Used')
plt.title('Linear Regression - Divvy Bike Usage')
plt.savefig('lin_reg_optimized.png')
"""

"\n# Graph results of the best models\n# Linear Gradient Boosting: max_samples=1.0, n_estimators=17\ngbr = GradientBoostingRegressor(random_state=0, max_samples=1.0, n_estimators=17)\ngbr.fit(X_train, y_train)\ny_pred = gbr.predict(X_test[1:2])\nprint(gbr.score(X_test, y_test))\n\nprint(np.sort(y_pred).shape)\n#print(np.sort(y, axis=0))\nplt.plot(np.arange(1,364), np.sort(y_pred, axis=0), 'o', label='original data')\ny_plot = np.sort(intercept[0] + coef[0][0]*x['weekend_or_holiday'] + coef[0][1]*x['humidity'] + coef[0][2]*x['temp'] + coef[0][3]*x['windspeed'])\nplt.plot(np.arange(1,364), y_plot, 'r', label='fitted line')\nplt.legend()\n#plt.show()\nplt.xlabel('Weekend/Holiday/Temperature Data')\nplt.ylabel('Number of Bikes Used')\nplt.title('Linear Regression - Divvy Bike Usage')\nplt.savefig('lin_reg_optimized.png')\n"

In [30]:
# Ridge and Lasso Regression
from sklearn.linear_model import Ridge, Lasso

def regression_optimizer(model, parameters, X_train, y_train, X_test, y_test):
  clf = GridSearchCV(model, parameters,cv=5)
  clf.fit(X_train,y_train)
  mean_test_score = clf.cv_results_['mean_test_score']
  best_params = clf.best_params_
  best_score = clf.best_score_
  # Find performance and coefficients for best model
  clf = Ridge(alpha=clf.best_params_['alpha'])
  clf.fit(X_train, y_train)
  clf.predict(X_test)
  test_score = clf.score(X_test, y_test)
  coef = clf.coef_
  intercept = clf.intercept_
  print(best_params)
  print(best_score)
  print(test_score)
  print(coef)
  print(intercept)
  return best_params, best_score, test_score, coef, intercept
   

print("Linear, Ridge")
#parameters = {'alpha':[0.1, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.05, 3.1, 3.15, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4, 5.0, 6, 7, 8, 9, 10]}
parameters = {'alpha':[0.1, 1, 3.41, 3.42, 3.43, 3.44, 3.45, 3.46, 3.57, 3.48, 3.49, 3.5, 3.51, 3.52, 3.53, 3.54, 3.55, 3.56, 3.57, 3.58, 3.59, 5, 10, 15, 100, 500, 1000]}
lr_best_params, lr_best_score, lr_test_score, lr_coef, lr_intercept = regression_optimizer(Ridge(), parameters, X_train, y_train, X_test, y_test)

print("Polynomial, Ridge")
#parameters = {'alpha': [50.0, 100, 200, 500, 800, 900, 1000, 1500, 2000, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3650, 3700, 3730, 3740, 3750, 3760, 3770, 3800]}
#parameters = {'alpha': [10,11,12,13,14,15,16,17,18,19,20,21,21.5,21.6,21.7,21.75,21.76,21.77,21.78,21.79,21.8,21.81,21.82,21.83,21.84,21.85,21.9,22,22.1,22.2,22.3,22.4,22.5,22.6,22.7,22.8,22.9,23,24,25,26,27,28,29]}
parameters = {'alpha': [0.1, 1, 5, 10, 19.1, 19.5, 19.7, 19.8, 19.9, 20, 20.1, 20.15, 20.2, 20.21, 20.22, 20.23, 20.24, 20.25, 20.26, 20.27, 20.28, 20.29, 20.3, 20.4, 20.5, 20.7, 21, 50, 100, 500, 1000]}
pr_best_params, pr_best_score, pr_test_score, pr_coef, lr_intercept = regression_optimizer(Ridge(), parameters, poly_features, y_train, X_test_features, y_test)

print("Linear, Lasso")
parameters = {'alpha':[0.01, 0.5, 0.6, 0.65, 0.7, 0.75, 0.8, 1.0, 1.05, 1.1, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.3, 1.35, 1.38, 1.4, 1.41, 1.42, 1.43, 1.44, 1.45, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 5, 10, 50, 100, 500, 1000]}
ll_best_params, ll_best_score, ll_test_score, ll_coef, lr_intercept = regression_optimizer(Lasso(), parameters, X_train, y_train, X_test, y_test)

print("Polynomial, Lasso")
#parameters = {'alpha':[10, 50, 60, 70, 80, 90, 100, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 120, 130, 140, 150, 200]}
parameters = {'alpha':[0.1, 0.5, 0.6, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.8, 0.9, 1, 1.5, 2, 3, 5, 5.5, 5.8, 6, 6.2, 6.5, 7,8,9,10,11,11.5,12,12.1,12.2,12.3,12.4,12.5,12.55,12.56,12.57,12.58,12.59,12.6,12.61,12.62,12.63,12.64,12.65,12.7,12.8,12.9,13,14, 50, 100, 500, 1000]}
pl_best_params, pl_best_score, pl_test_score, pl_coef, lr_intercept = regression_optimizer(Lasso(), parameters, poly_features, y_train, X_test_features, y_test)



Linear, Ridge
{'alpha': 3.51}
0.5436666976816236
0.5045120093446303
[[198.65743672  15.16343273  -6.6728615 ]]
[224.93375222]
Polynomial, Ridge
{'alpha': 20.23}
0.6280984158272822
0.6351032853540073
[[-4.37178073  7.7113819  47.4477518  -4.37178073  6.69223301 -1.89916517
   0.17982763 -0.17049044 -0.34973373]]
[-1402.16705947]
Linear, Lasso
{'alpha': 1.23}
0.5435433510637158
0.5066836542633393
[[205.7533736   15.16318442  -6.68970199]]
[223.76340485]
Polynomial, Lasso


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.414e+06, tolerance: 6.478e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.499e+06, tolerance: 6.501e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

{'alpha': 0.67}
0.6281249579646332
0.6361606736516094
[[-25.42540717   8.09473786  49.63280324 -25.42540717   6.81941858
   -1.35767233   0.17904886  -0.17597109  -0.36582018]]
[-1473.20979887]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.340e+05, tolerance: 8.193e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
